<a href="https://colab.research.google.com/github/mengwangk/dl-projects/blob/master/04_03_auto_ml_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated ML

In [0]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2'

In [2]:
if COLAB:
  # !sudo apt-get install git-lfs && git lfs install
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  #!cd dl-projects && ls -l --block-size=M

Cloning into 'dl-projects'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 831 (delta 29), reused 2 (delta 1), pack-reused 781
Receiving objects: 100% (831/831), 68.80 MiB | 24.52 MiB/s, done.
Resolving deltas: 100% (477/477), done.


In [0]:
if COLAB:
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .

In [0]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
import math 
import matplotlib

from scipy import stats
from collections import Counter
from pathlib import Path

plt.style.use('fivethirtyeight')

sns.set(style="ticks")

# Automated feature engineering
import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, precision_recall_curve, roc_curve, mean_squared_error, accuracy_score, average_precision_score
from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn.ensemble import RandomForestClassifier
#from scikitplot.plotters import plot_precision_recall_curve

from IPython.display import display

from utils import *
from preprocess import *

# The Answer to the Ultimate Question of Life, the Universe, and Everything.
np.random.seed(42)

In [6]:
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



## Preparation

In [7]:
if COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
if COLAB:
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
else:
  DATASET_PATH = Path("datasets")
  ORIGIN_DATASET_PATH = Path('datasets')

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.pkl"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

if COLAB:
  !ls -l gdrive/"My Drive"/datasets/ --block-size=M
  !ls -l dl-projects/datasets --block-size=M

total 3954M
-rw------- 1 root root  407M Dec 30 05:00 feature_matrix_2.csv
-rw------- 1 root root  428M Dec 30 05:01 feature_matrix_2.pkl
-rw------- 1 root root  141M Dec 27 08:27 feature_matrix.csv
-rw------- 1 root root 1637M Jan  4 07:31 feature_matrix_d2.csv
-rw------- 1 root root 1166M Jan  4 07:35 feature_matrix_d2.pkl
-rw------- 1 root root  176M Dec 27 08:28 feature_matrix.pkl
total 1M
-rw-r--r-- 1 root root 1M Jan  4 15:49 4D.zip


In [0]:
#data = pd.read_csv(DATASET, header=0, sep=',', quotechar='"', parse_dates=['time'])
data = pd.read_pickle(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719972 entries, 0 to 719971
Columns: 221 entries, NumberId to year
dtypes: datetime64[ns](1), float64(146), int64(64), uint8(10)
memory usage: 1.1 GB


## Exploratory Data Analysis

In [0]:
feature_matrix = data

In [12]:
display(feature_matrix.columns)

Index(['NumberId', 'time', 'STD(Results.DrawNo)', 'STD(Results.LuckyNo)',
       'STD(Results.TotalStrike)', 'MAX(Results.DrawNo)',
       'MAX(Results.LuckyNo)', 'MAX(Results.TotalStrike)',
       'MIN(Results.DrawNo)', 'MIN(Results.LuckyNo)',
       ...
       'MODE(Results.PrizeType)_3rdPrizeNo',
       'MODE(Results.PrizeType)_Consolation',
       'MODE(Results.PrizeType)_Special', 'LAST(Results.PrizeType)_1stPrizeNo',
       'LAST(Results.PrizeType)_2ndPrizeNo',
       'LAST(Results.PrizeType)_3rdPrizeNo',
       'LAST(Results.PrizeType)_Consolation',
       'LAST(Results.PrizeType)_Special', 'month', 'year'],
      dtype='object', length=221)

In [13]:
display(feature_matrix.head(4))

,NumberId,time,STD(Results.DrawNo),STD(Results.LuckyNo),STD(Results.TotalStrike),MAX(Results.DrawNo),MAX(Results.LuckyNo),MAX(Results.TotalStrike),MIN(Results.DrawNo),MIN(Results.LuckyNo),MIN(Results.TotalStrike),MEAN(Results.DrawNo),MEAN(Results.LuckyNo),MEAN(Results.TotalStrike),SKEW(Results.DrawNo),SKEW(Results.LuckyNo),SKEW(Results.TotalStrike),LAST(Results.DrawNo),LAST(Results.LuckyNo),LAST(Results.TotalStrike),LAST(Results.results_index),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.DrawNo),SUM(Results.LuckyNo),SUM(Results.TotalStrike),"TREND(Results.DrawNo, DrawDate)","TREND(Results.LuckyNo, DrawDate)","TREND(Results.TotalStrike, DrawDate)",NUM_UNIQUE(Results.PrizeType),DAY(first_Results_time),MONTH(first_Results_time),HOUR(first_Results_time),TIME_SINCE(first_Results_time),STD(Results.PERCENTILE(TotalStrike)),STD(Results.CUM_MEAN(LuckyNo)),STD(Results.CUM_SUM(TotalStrike)),STD(Results.CUM_SUM(DrawNo)),STD(Results.CUM_SUM(LuckyNo)),STD(Results.PERCENTILE(DrawNo)),...,PERCENTILE(SUM(Results.DrawNo)),PERCENTILE(SKEW(Results.DrawNo)),PERCENTILE(MAX(Results.DrawNo)),PERCENTILE(SUM(Results.TotalStrike)),PERCENTILE(MAX(Results.TotalStrike)),"PERCENTILE(TREND(Results.LuckyNo, DrawDate))",PERCENTILE(MAX(Results.LuckyNo)),PERCENTILE(NUM_UNIQUE(Results.PrizeType)),PERCENTILE(LAST(Results.DrawNo)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",PERCENTILE(MEAN(Results.DrawNo)),PERCENTILE(MIN(Results.DrawNo)),PERCENTILE(MEAN(Results.LuckyNo)),PERCENTILE(SUM(Results.LuckyNo)),PERCENTILE(COUNT(Results)),PERCENTILE(LAST(Results.TotalStrike)),"PERCENTILE(TREND(Results.TotalStrike, DrawDate))",PERCENTILE(MIN(Results.TotalStrike)),PERCENTILE(LAST(Results.LuckyNo)),PERCENTILE(SKEW(Results.TotalStrike)),PERCENTILE(MIN(Results.LuckyNo)),PERCENTILE(STD(Results.DrawNo)),PERCENTILE(MEAN(Results.TotalStrike)),PERCENTILE(STD(Results.LuckyNo)),PERCENTILE(AVG_TIME_BETWEEN(Results.DrawDate)),PERCENTILE(STD(Results.TotalStrike)),TotalStrike,Label,MODE(Results.PrizeType)_1stPrizeNo,MODE(Results.PrizeType)_2ndPrizeNo,MODE(Results.PrizeType)_3rdPrizeNo,MODE(Results.PrizeType)_Consolation,MODE(Results.PrizeType)_Special,LAST(Results.PrizeType)_1stPrizeNo,LAST(Results.PrizeType)_2ndPrizeNo,LAST(Results.PrizeType)_3rdPrizeNo,LAST(Results.PrizeType)_Consolation,LAST(Results.PrizeType)_Special,month,year
0,45,2014-01-01,112624.014707,0.0,0.0,380012,45,1,97695,45,1,279760.909091,45,1,-0.930399,0.0,0.0,380012,45,1,78016,5.347296e+07,11,3077370,495,11,45.560053,0.0,0.0,3,23,12,0,568857600.0,0.0,5.101631,25901.838735,6.189125e+09,1.288990e+08,0.314047,...,0.954191,0.088030,0.641928,0.826365,0.50005,0.500051,0.004601,0.580366,0.641928,0.751189,0.939988,0.732947,0.004601,0.008102,0.826365,0.50005,0.500051,0.50005,0.004601,0.500051,0.004601,0.701934,0.50005,0.50005,0.170524,0.50005,1.0,1,0,0,0,1,0,1,0,0,0,0,1,2014
1,64,2014-01-01,120519.321745,0.0,0.0,375112,64,1,61593,64,1,230353.500000,64,1,-0.167286,0.0,0.0,375112,64,1,76900,1.198022e+08,6,1382121,384,6,45.316013,0.0,0.0,3,2,9,0,641606400.0,0.0,8.481822,27713.008869,6.313569e+09,1.379830e+08,0.336035,...,0.285457,0.389659,0.572615,0.226895,0.50005,0.500051,0.006501,0.580366,0.572615,0.567641,0.609822,0.378876,0.006501,0.006501,0.226895,0.50005,0.500051,0.50005,0.006501,0.500051,0.006501,0.826070,0.50005,0.50005,0.884080,0.50005,1.0,1,0,0,0,0,1,0,0,0,1,0,1,2014
2,81,2014-01-01,113967.559506,0.0,0.0,358811,81,1,41792,81,1,211827.500000,81,1,-0.543717,0.0,0.0,358811,81,1,73156,8.715291e+07,8,1694620,648,8,44.679995,0.0,0.0,2,28,5,0,681523200.0,0.0,31.435416,26209.269549,5.083894e+09,1.304944e+08,0.317785,...,0.458742,0.198017,0.397880,0.490798,0.50005,0.500051,0.008202,0.192488,0.397880,0.129313,0.415433,0.023955,0.008202,0.010202,0.490798,0.50005,0.500051,0.50005,0.008202,0.500051,0.008202,0.726280,0.50005,0.50005,0.678690,0.50005,1.0,1,0,0,0,1,0,0,0,0,0,1,1,2014
3,93,2014-01-01,127901.309529,0.0,0.0,363111,93,1,41892,93,1,181780.400000,93,1,0.469604,0.0,0.0,363111,93,1,74151,1.543536e+08,5,908902,4

In [14]:
origin_data[origin_data['LuckyNo']==911].head(10)

,DrawNo,DrawDate,PrizeType,LuckyNo
10287,85495,1995-03-12,ConsolationNo3,911
13607,99896,1996-02-11,SpecialNo10,911
26564,156199,1999-08-29,SpecialNo9,911
30160,171800,2000-08-16,ConsolationNo4,911
65817,326909,2009-12-05,SpecialNo10,911
89716,430815,2015-10-07,SpecialNo3,911
96458,460117,2017-05-27,SpecialNo6,911
102733,487418,2018-12-08,SpecialNo2,911
104575,495419,2019-06-01,SpecialNo4,911
104910,496919,2019-07-03,ConsolationNo4,911


In [0]:
# feature_matrix.groupby('time')['COUNT(Results)'].mean().plot()
# plt.title('Average Monthly Count of Results')
# plt.ylabel('Strike Per Number')

## Feature Selection

In [16]:
from utils import feature_selection

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
feature_matrix_selection = feature_selection(feature_matrix.drop(columns = ['time', 'NumberId']))

Original shape:  (719972, 219)
0 missing columns with threshold: 90.
42 zero variance columns.


In [0]:
feature_matrix_selection['time'] = feature_matrix['time']
feature_matrix_selection['NumberId'] = feature_matrix['NumberId']
feature_matrix_selection['Label'] = feature_matrix['Label']

In [0]:
feature_matrix_selection.columns

## Correlations

In [0]:
feature_matrix_selection.shape

In [0]:
corrs = feature_matrix_selection.corr().sort_values('TotalStrike')
corrs['TotalStrike'].head()

In [0]:
corrs['Label'].dropna().tail(15)

In [0]:
corrs['TotalStrike'].dropna().tail(8)

## Visualization

In [0]:
#pip install autoviz

In [0]:
#from autoviz.AutoViz_Class import AutoViz_Class

### XgBoost

In [0]:
import xgboost as xgb

In [0]:
model = xgb.XGBClassifier()

In [0]:
def predict_dt(dt, feature_matrix, return_probs = False):

    feature_matrix['date'] = feature_matrix['time']

    # Subset labels
    test_labels = feature_matrix.loc[feature_matrix['date'] == dt, 'Label']
    train_labels = feature_matrix.loc[feature_matrix['date'] < dt, 'Label']

    print(f"Size of test labels {len(test_labels)}")
    print(f"Size of train labels {len(train_labels)}")
    
    # Features
    X_train = feature_matrix[feature_matrix['date'] < dt].drop(columns = ['NumberId', 'time',
                                                                                     'date', 'Label', 'TotalStrike', 'month', 'year', 'index'], errors='ignore')
    X_test = feature_matrix[feature_matrix['date'] == dt].drop(columns = ['NumberId', 'time',
                                                                                     'date', 'Label', 'TotalStrike', 'month', 'year', 'index'], errors='ignore')
    print(f"Size of X train {len(X_train)}")
    print(f"Size of X test  {len(X_test)}")
   
    feature_names = list(X_train.columns)
    
    # Impute and scale features
    pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'median')), 
                      ('scaler', MinMaxScaler())])

    # Fit and transform training data
    X_train = pipeline.fit_transform(X_train)
    X_test = pipeline.transform(X_test)
    
    # Labels
    y_train = np.array(train_labels).reshape((-1, ))
    y_test = np.array(test_labels).reshape((-1, ))
    
    print('Training on {} observations.'.format(len(X_train)))
    print('Testing on {} observations.\n'.format(len(X_test)))
    
    # Train 
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    probs = model.predict_proba(X_test)[:, 1]
    
    # Total positive
    positive = np.where((predictions==1))
    print('Total predicted to be positive: ', len(positive[0]))
  
    # Calculate metrics
    p_score = precision_score(y_test, predictions)
    r_score = recall_score(y_test, predictions)
    f_score = f1_score(y_test, predictions)
    auc_score = roc_auc_score(y_test, probs)
    a_score = accuracy_score(y_test, predictions)
    avg_p_score = average_precision_score(y_test, predictions)
    cm = confusion_matrix(y_test, predictions)
    
    print(f'Precision: {round(p_score, 5)}')
    print(f'Recall: {round(r_score, 5)}')
    print(f'F1 Score: {round(f_score, 5)}')
    print(f'ROC AUC: {round(auc_score, 5)}')
    print(f'Accuracy: {round(a_score, 5)}')
    print(f'Average precision: {round(avg_p_score, 5)}')
    
    print('Confusion matrix')
    print(cm)

    # Total predicted matches
    print('Predicted matches')
    m = np.where((predictions==1))
    print(len(m[0]), m)

    if len(positive[0]) > 0:
      # Matching draws
      print('Matched draws')
      m = np.where((predictions==1) & (y_test == 1))
      print(len(m[0]), m)
      month_data = feature_matrix.loc[feature_matrix['date'] == dt]
      numbers = month_data.iloc[m[0]][['NumberId']]
      print(numbers)
      display(origin_data[(origin_data['DrawDate'].dt.year == dt.year) & 
                          (origin_data['DrawDate'].dt.month == dt.month) & 
                          (origin_data['LuckyNo'].isin(numbers))].head(len(positive[0])))                                                   
    else:
      print('No luck this month')                 

    # Feature importances
    fi = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})
    
    if return_probs:
        return fi, probs
    
    return fi
    

In [0]:
# All the months
len(feature_matrix_selection['time'].unique()), feature_matrix_selection['time'].unique()

### Prediction by months

In [0]:
from utils import plot_feature_importances

In [0]:
%time june_2019 = predict_dt(pd.datetime(2019,6,1), feature_matrix_selection)

In [0]:
norm_june_2019_fi = plot_feature_importances(june_2019)

In [0]:
 # %time oct_2019 = predict_dt(pd.datetime(2019,10,1), feature_matrix_selection)

## Parameter Tuning - GridSearchCV

## Check Raw Data

In [0]:
display(origin_data[(origin_data['DrawDate'].dt.year == 2019) & (origin_data['DrawDate'].dt.month == 10) & (origin_data['LuckyNo'].isin([3162]))].head(10))

In [0]:
origin_data[(origin_data['DrawDate'].dt.year == 2019) & (origin_data['DrawDate'].dt.month == 6)]['DrawNo'].nunique()

In [0]:
origin_data[(origin_data['DrawDate'].dt.year == 2019) & (origin_data['DrawDate'].dt.month == 10)]['DrawNo'].nunique()

In [0]:
print(15 * 23 + 14 * 23)